## RouterAgent

智能体路由可以扩展为更广泛的场景，比如根据不同的工具集合、向量库类型等，配置为不同的智能体，
然后根据用户的意图和场景，使用 RouterAgent 动态选择最佳的智能体。

RouterAgent 有两个参数：
- condition 选择函数，可以是普通函数、 lambda 函数，或者实现了 `__call__` 方法的对象
    - condition 接受两个参数，一是提示语，二是智能体名称列表
- agents 智能体列表

### 基础示范

In [3]:
from illufly.types import RouterAgent
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM

def myfunc(agents, consumer_dict):
    return agents[0]

agent = RouterAgent(myfunc, [FakeLLM(), ChatQwen(), ChatZhipu()])
agent.bind_provider({"task": "写一首歌"})

agent("你是什么模型？")

这是一个模拟调用!

'这是一个模拟调用!'

In [4]:
agent.selected

<FakeLLM.4425535632>

### 路由 ChatAgent

考虑到你有两个专业的 ChatAgent:

- data_agent 擅长数据分析
- writer_agent 擅长写作

你已经根据为他们分别配置了提示语模板、工具集、数据库等，但现在你需要根据用户的意图匹配到底使用哪一个智能体。<br>

那么如何判断用户意图呢？
- 方案1 可以使用大模型来做甄别，这会增加一个环节，推理过程可能延长用户的等待
- 方案2 可以根据问题做文本相似性比较，这样做效率很高

我们的例子中探讨方案2如何实现，关键就是与什么做比较？

原始的可比较信息是：用户问题与智能体描述文本之间的比较，但这明显不太够用。<br>
可以使用**检索器**和**经验缓存**来弥补。


In [8]:
agent.consumer_dict

{'task': '写一首歌'}

### 路由 Template

In [17]:
from illufly.types import Template, RouterAgent

r = RouterAgent(
    condition=lambda runs, vars: runs[0],
    runnables=[Template(text="请帮我：{{task}}"), Template(text="你是一个作家")]
)
r.bind_provider({"task": "写一首歌"})

r.selected

<Template consumer_dict={'task'} text='请帮我：{{task}}'>

使用 lazy_binding_map 可以先声明对象，然后在合适的时机自动实现绑定。

In [16]:
from illufly.chat import ChatQwen
from illufly.types import Template, RouterAgent

t = RouterAgent(
    lambda runs, vars: runs[1], 
    [
        Template(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        Template(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ]
)
qwen = ChatQwen(memory=[t])
qwen("小兔子")

你可以为你的小兔子起名"兔小宝"，这个名字既可爱又富有亲和力。


'你可以为你的小兔子起名"兔小宝"，这个名字既可爱又富有亲和力。'

In [13]:
t.provider_dict

{'last_output': '(一)\n天空的云飘过，心中有无数话要说，\n岁月如流水，带走了青春的梦想。\n每一天的日出，唤醒我沉睡的灵魂，\n让我继续向前走，不再回头。\n\n(二)\n人生的路漫长，有时候会感到孤单，\n但我知道，我不是一个人在战斗。\n每一步的脚印，都是我成长的见证，\n让我更加坚强，不再害怕跌倒。\n\n(副歌)\n我要唱一首歌，为那些曾经的挫折，\n为那些曾经的努力，为那些曾经的自己。\n我要唱一首歌，为那些未来的希望，\n为那些未来的梦想，为那些未来的自己。\n\n(桥段)\n无论前方有多少困难，我都会勇敢地面对，\n因为我知道，只有这样，才能让自己变得更好。\n我会一直坚持下去，直到最后的胜利，\n因为我相信，只要努力，就一定能够实现梦想。\n\n(副歌)\n我要唱一首歌，为那些曾经的挫折，\n为那些曾经的努力，为那些曾经的自己。\n我要唱一首歌，为那些未来的希望，\n为那些未来的梦想，为那些未来的自己。\n\n(尾声)\n我要唱一首歌，为自己，\n为自己加油，为自己打气，\n为自己唱一首歌。',
 'resources': '',
 'task': '写一首歌',
 'tools_name': '',
 'tools_desc': '',
 'knowledge': []}

### 路由 Retrieval

### 路由 VectorDB